In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
grid_len = 3
grid_size = grid_len**2
def check_three(state, arr, value):
    if state[arr[0]] == state[arr[1]] and state[arr[1]] == state[arr[2]] and state[arr[2]] == value:
        return True
    return False
def is_winner(state):
    return check_three(state, [0, 4, 8], 1) or check_three(state, [2, 4, 6], 1) or check_three(state, [0, 1, 2], 1)\
                or check_three(state, [3, 4, 5], 1) or check_three(state, [6, 7, 8], 1) or check_three(state, [0, 3, 6], 1)\
                or check_three(state, [1, 4, 7], 1) or check_three(state, [2, 5, 8], 1)
def is_loser(state):
    return check_three(state, [0, 4, 8], -1) or check_three(state, [2, 4, 6], -1) or check_three(state, [0, 1, 2], -1)\
                or check_three(state, [3, 4, 5], -1) or check_three(state, [6, 7, 8], -1) or check_three(state, [0, 3, 6], -1)\
                or check_three(state, [1, 4, 7], -1) or check_three(state, [2, 5, 8], -1)
def get_reward(state):
    if is_winner(state):
        return 1
    elif is_loser(state):
        return -1
    else:
        return 0.1

In [126]:
def get_reward_for_action(state, index):
    s = state[0]
#     with tf.Session() as sess2:
#         s = sess2.run(state[0])
    if s[index] != 0:
        retuen -3
    else:
        return get_reward(s)

In [ ]:
# 12 x 24 x 36 x 18 x 9 -> softmax

In [139]:
tf.reset_default_graph()

def check_three(state, arr, value):
    if state[arr[0]] == state[arr[1]] and state[arr[1]] == state[arr[2]] and state[arr[2]] == value:
        return True
    return False
def is_winner(state):
    return check_three(state, [0, 4, 8], 1) or check_three(state, [2, 4, 6], 1) or check_three(state, [0, 1, 2], 1)\
                or check_three(state, [3, 4, 5], 1) or check_three(state, [6, 7, 8], 1) or check_three(state, [0, 3, 6], 1)\
                or check_three(state, [1, 4, 7], 1) or check_three(state, [2, 5, 8], 1)
def is_loser(state):
    return check_three(state, [0, 4, 8], -1) or check_three(state, [2, 4, 6], -1) or check_three(state, [0, 1, 2], -1)\
                or check_three(state, [3, 4, 5], -1) or check_three(state, [6, 7, 8], -1) or check_three(state, [0, 3, 6], -1)\
                or check_three(state, [1, 4, 7], -1) or check_three(state, [2, 5, 8], -1)
def get_reward(state):
    if is_winner(state):
        return tf.constant(1, dtype=tf.float32)
    elif is_loser(state):
        return tf.constant(-1, dtype=tf.float32)
    else:
        return tf.constant(0.1, dtype=tf.float32)

def get_reward_for_action(state, index):
    s = state[0]
    print(s.shape, index.shape)
    if s[index] != 0:
        return tf.constant(-3, dtype=tf.float32)
    else:
        return get_reward(s)
    
X = tf.placeholder(tf.float32, shape=[1, 9], name="X")

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

#first layer
with tf.variable_scope('first_hidden') as scope:
    X2 = tf.reshape(X, shape=[-1, 9])
    W1 = tf.get_variable("weights", shape=[9, 12], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable('bias', shape=[12], initializer=tf.constant_initializer(0.0))
    z1 = tf.matmul(X2, W1) + b1
    a1 = tf.nn.relu(z1, name="a1")
    
    
#second layer
with tf.variable_scope('second_hidden') as scope:
    W2 = tf.get_variable("weights", shape=[12, 24], initializer=tf.truncated_normal_initializer())
    b2 = tf.get_variable('bias', shape=[24], initializer=tf.constant_initializer(0.0))
    z2 = tf.matmul(a1, W2) + b2
    a2 = tf.nn.relu(z2, name="a2")
    
#third layer
with tf.variable_scope('third_hidden') as scope:
    W3 = tf.get_variable("weights", shape=[24, 18], initializer=tf.truncated_normal_initializer())
    b3 = tf.get_variable('bias', shape=[18], initializer=tf.constant_initializer(0.0))
    z3 = tf.add(tf.matmul(a2, W3), b3)
    a3 = tf.nn.relu(z3, name="a3")
    
    
#fourth layer
with tf.variable_scope('fourth_hidden') as scope:
    W5 = tf.get_variable("weights", shape=[18, 9], initializer=tf.truncated_normal_initializer())
    b5 = tf.get_variable('bias', shape=[9], initializer=tf.constant_initializer(0.0))
    z5 = tf.add(tf.matmul(a3, W5), b5)
    a5 = tf.nn.relu(z5, name="a5")
    
print(a5.shape)
a6 = tf.argmax(a5[0])

loss = get_reward_for_action(X2, a6)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

with tf.name_scope('summary'):
    tf.summary.scalar('loss', loss)
    summary_op = tf.summary.merge_all()

states = []
actions = []
with tf.Session() as sess:
    tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    writer = tf.summary.FileWriter('./my_graph/tic', sess.graph)
    
    initial_step = global_step.eval()
    
    total_loss = 0.0
    for i in range(1000):
        new_state = np.random.randint(-1, 1, size=[1, 9])
        states.append(new_state)
        _, l, ac, summary = sess.run([optimizer, loss, a6, summary_op], feed_dict={X: new_state})
        actions.append(ac)
        total_loss += l
        writer.add_summary(summary, global_step=i)
        if i % 10 == 0:
            print('loss at step', str(i), '=', str(l), 'with state', str(new_state), 'and action', str(ac))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/tic/tic', i)

(1, 9)
(9,) ()


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'first_hidden/weights:0' shape=(9, 12) dtype=float32_ref>", "<tf.Variable 'first_hidden/bias:0' shape=(12,) dtype=float32_ref>", "<tf.Variable 'second_hidden/weights:0' shape=(12, 24) dtype=float32_ref>", "<tf.Variable 'second_hidden/bias:0' shape=(24,) dtype=float32_ref>", "<tf.Variable 'third_hidden/weights:0' shape=(24, 18) dtype=float32_ref>", "<tf.Variable 'third_hidden/bias:0' shape=(18,) dtype=float32_ref>", "<tf.Variable 'fourth_hidden/weights:0' shape=(18, 9) dtype=float32_ref>", "<tf.Variable 'fourth_hidden/bias:0' shape=(9,) dtype=float32_ref>"] and loss Tensor("Const_1:0", shape=(), dtype=float32).